In [30]:
import pandas as pd

In [31]:
years= list(range(1999,2024))

In [32]:
csv_files=[f"./Data/play_by_play_{year}.csv" for year in years]

In [33]:
# load each csv file as a dataframe and collect them in a list
dataframes= []
for csv_file in csv_files:
    df= pd.read_csv(csv_file)
    dataframes.append(df)


C:\Users\thilo\AppData\Local\Temp\ipykernel_15088\3488205839.py:4: DtypeWarning: Columns (36,37,179,180,189,190,197,198,203,204,205,206,213,214,218,219,220,222,224,226,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(csv_file)
C:\Users\thilo\AppData\Local\Temp\ipykernel_15088\3488205839.py:4: DtypeWarning: Columns (37,45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,218,219,220,248,249,253,254,255,260,262,263,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(csv_file)
C:\Users\thilo\AppData\Local\Temp\ipykernel_15088\3488205839.py:4: DtypeWarning: Columns (37,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,248,249,253,254,255,260,262,263,266,267,268,269,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(csv_file)
C:\Users\thilo\AppData\Loc

In [34]:
list(dataframes[0]["play_type"].unique())

['kickoff',
 'pass',
 'run',
 'no_play',
 'punt',
 'extra_point',
 nan,
 'field_goal',
 'qb_kneel',
 'qb_spike']

In [35]:
# remove useless plays from dataframes to reduce size for combining dataframes
for dataframe in dataframes:
    dataframe.drop(dataframe[~dataframe['play_type'].isin(["pass", "run"])].index, axis=0, inplace=True)

In [36]:
# combine all dataframes into a single one
combined_df = pd.concat(dataframes, axis=0)

# reset the index of the combined dataframe
combined_df.reset_index(drop=True, inplace=True)



In [37]:
# drop safeties
combined_df.drop(combined_df[combined_df["safety"]==1].index, axis=0, inplace=True)

In [38]:
# drop plays with penalties
combined_df.drop(combined_df[combined_df["penalty"]==1].index, axis=0, inplace=True)

In [39]:
# drop plays with laterals
combined_df.drop(combined_df[combined_df["lateral_reception"]==1].index, axis=0, inplace=True)

In [40]:
# drop plays with laterals
combined_df.drop(combined_df[combined_df["lateral_rush"]==1].index, axis=0, inplace=True)

In [41]:
# drop plays with replays or challenges
combined_df.drop(combined_df[combined_df["replay_or_challenge"]==1].index, axis=0, inplace=True)

In [ ]:
#  make copy of combined_df
combined_df_copy= combined_df.copy()

In [43]:
drop_columns1 = [
    "play_id",
    "game_id",
    "old_game_id",
    "home_team",
    "away_team",
    "season_type",
    "week",
    "defteam",
    "side_of_field",
    #"game_date",
    "quarter_seconds_remaining",
    "quarter_end",
    "drive",
    "sp",  # eventuell relevant laut TOK
    "qtr",
    "time",
    "yrdln",
    "ydsnet",
    "qb_kneel",
    "qb_spike",
    "pass_length",
    "pass_location",
    "run_location",
    "run_gap",
    "field_goal_result",
    "kick_distance",
    "extra_point_result",
    "two_point_conv_result",
    "home_timeouts_remaining",
    "away_timeouts_remaining",
    "timeout",
    "timeout_team",
    "td_team",
    "td_player_name",
    "td_player_id",
    "posteam_timeouts_remaining",
    "defteam_timeouts_remaining",
    "total_home_score",
    "total_away_score",
    "posteam_score",
    "defteam_score",
    "posteam_score_post",
    "defteam_score_post",
    "score_differential_post",
    "no_score_prob",
    "opp_fg_prob",
    "opp_safety_prob",
    "opp_td_prob",
    "fg_prob",
    "safety_prob",
    "extra_point_prob",
    "two_point_conversion_prob",
    "total_home_epa",
    "total_away_epa",
    "total_home_rush_epa",
    "total_away_rush_epa",
    "total_home_pass_epa",
    "total_away_pass_epa",
    "air_epa",
    "yac_epa",
    "comp_air_epa",
    "comp_yac_epa",
    "total_home_comp_air_epa",
    "total_away_comp_air_epa",
    "total_home_comp_yac_epa",
    "total_away_comp_yac_epa",
    "total_home_raw_air_epa",
    "total_away_raw_air_epa",
    "total_home_raw_yac_epa",
    "total_away_raw_yac_epa",
    "def_wp",
    "home_wp",
    "away_wp",
    "vegas_wpa",
    "vegas_home_wpa",
    "home_wp_post",
    "away_wp_post",
    "vegas_wp",
    "vegas_home_wp",
    "total_home_rush_wpa",
    "total_away_rush_wpa",
    "total_home_pass_wpa",
    "total_away_pass_wpa",
    "air_wpa",
    "yac_wpa",
    "comp_air_wpa",
    "comp_yac_wpa",
    "total_home_comp_air_wpa",
    "total_away_comp_air_wpa",
    "total_home_comp_yac_wpa",
    "total_away_comp_yac_wpa",
    "total_home_raw_air_wpa",
    "total_away_raw_air_wpa",
    "total_home_raw_yac_wpa",
    "total_away_raw_yac_wpa",
    "punt_blocked",
    "touchback",
    
]

In [44]:
drop_columns2 = [
    "punt_attempt",
    "safety_player_name",
    "punt_inside_twenty",
    "kicker_player_name",
    "passing_yards",
    "interception_player_name",
    "lateral_kickoff_returner_player_id",
    "assist_tackle",
    "qb_hit_2_player_id",
    "penalty_team",
    "lateral_receiver_player_name",
    "lateral_recovery",
    "wind",
    "rush_touchdown",
    "defensive_two_point_conv",
    "receiver_jersey_number",
    "tackle_with_assist",
    "fumbled_1_player_name",
    "tackle_with_assist_2_player_id",
    "two_point_attempt",
    "drive_quarter_start",
    "lateral_receiving_yards",
    "end_yard_line",
    "defensive_extra_point_attempt",
    "pass_touchdown",
    "lateral_rusher_player_name",
    "rusher_id",
    "aborted_play",
    "drive_yards_penalized",
    "fumble_not_forced",
    "penalty_player_id",
    "tackle_for_loss_2_player_id",
    "drive_end_yard_line",
    "passer_id",
    "stadium_id",
    "sack_player_name",
    "punt_out_of_bounds",
    "tackle_with_assist_2_team",
    "surface",
    "kickoff_downed",
    "extra_point_attempt",
    "punt_fair_catch",
    "assist_tackle_2_player_name",
    "fumble_forced",
    "special_teams_play",
    "drive_ended_with_score",
    "half_sack_1_player_name",
    "stadium",
    "lateral_receiver_player_id",
    "lateral_sack_player_name",
    "play",
    "tackle_with_assist_1_player_id",
    "forced_fumble_player_2_team",
    "home_coach",
    "xyac_epa",
    "lateral_punt_returner_player_name",
    "fantasy",
    "solo_tackle_1_team",
    "drive_start_transition",
    "first_down",
    "game_stadium",
    "xyac_fd",
    "drive_play_count",
    "passer",
    "fumbled_1_player_id",
    "replay_or_challenge_result",
    "drive_real_start_time",
    "receiver_player_id",
    "solo_tackle_2_player_id",
    "fumbled_2_player_name",
    "qb_hit_1_player_name",
    "kickoff_attempt",
    "xyac_success",
    "season",
    "rush",
    "tackle_with_assist_2_player_name",
    "assist_tackle_2_team",
    "sack_player_id",
    "assist_tackle_1_team",
    "play_deleted",
    "rusher_jersey_number",
    "pass_oe",
    "return_team",
    "tackle_for_loss_2_player_name",
    "time_of_day",
    "end_clock_time",
    "tackle_with_assist_1_team",
    "home_score",
    "tackle_with_assist_1_player_name",
    "kickoff_inside_twenty",
    "own_kickoff_recovery_player_id",
    "lateral_reception",
    "qb_hit_1_player_id",
    "own_kickoff_recovery_td",
    "pass_defense_2_player_name",
    "jersey_number",
    "punter_player_name",
    "blocked_player_name",
    "pass_defense_1_player_name",
    "xyac_median_yardage",
    "st_play_type",
    "success",
    "penalty_player_name",
    "punt_returner_player_name",
    "return_touchdown",
    "blocked_player_id",
    "assist_tackle_1_player_id",
    "receiving_yards",
    "half_sack_2_player_name",
    "drive_game_clock_start",
    "rusher",
    "pass_defense_1_player_id",
    "touchdown",
    "assist_tackle_4_player_id",
    "lateral_return",
    "solo_tackle_2_team",
    "kickoff_in_endzone",
    "fumble_out_of_bounds",
    "return_yards",
    "punt_downed",
    "nfl_api_id",
    "defensive_extra_point_conv",
    "out_of_bounds",
    "lateral_interception_player_name",
    "lateral_rush",
    "interception_player_id",
    "assist_tackle_3_player_name",
    "pass_defense_2_player_id",
    "receiver_player_name",
    "away_score",
    "forced_fumble_player_2_player_name",
    "qb_hit_2_player_name",
    "order_sequence",
    "lateral_rusher_player_id",
    "punt_returner_player_id",
    "play_clock",
    "cpoe",
    "punt_in_endzone",
    "fantasy_player_name",
    "passer_player_name",
    "xyac_mean_yardage",
    "fixed_drive",
    "forced_fumble_player_1_player_name",
    "lateral_interception_player_id",
    "solo_tackle",
    "kickoff_out_of_bounds",
    "fumbled_2_player_id",
    "fumbled_1_team",
    "defensive_two_point_attempt",
    "spread_line",
    "drive_game_clock_end",
    "home_opening_kickoff",
    "fantasy_id",
    "forced_fumble_player_1_player_id",
    "away_coach",
    "fumbled_2_team",
    "kickoff_fair_catch",
    "half_sack_1_player_id",
    "receiver",
    "punter_player_id",
    "xpass",
    "replay_or_challenge",
    "rusher_player_name",
    "pass",
    "assist_tackle_1_player_name",
    "fixed_drive_result",
    "kickoff_returner_player_id",
    "forced_fumble_player_1_team",
    "half_sack_2_player_id",
    "temp",
    "id",
    "drive_end_transition",
    "passer_player_id",
    "tackle_for_loss_1_player_name",
    "field_goal_attempt",
    "lateral_punt_returner_player_id",
    "play_type_nfl",
    "drive_first_downs",
    "result",
    "receiver_id",
    "start_time",
    "name",
    "rusher_player_id",
    "passer_jersey_number",
    "fantasy_player_id",
    "tackle_for_loss_1_player_id",
    "own_kickoff_recovery_player_name",
    "drive_time_of_possession",
    "forced_fumble_player_2_player_id",
    "assist_tackle_2_player_id",
    "own_kickoff_recovery",
    "solo_tackle_1_player_name",
    "special",
    "lateral_sack_player_id",
    "lateral_rushing_yards",
    "assist_tackle_3_team",
    "drive_quarter_end",
    "location",
    "total",
    "rushing_yards",
    "total_line",
    "solo_tackle_1_player_id",
    "assist_tackle_4_player_name",
    "assist_tackle_4_team",
    "safety_player_id",
    "drive_start_yard_line",
    "kicker_player_id",
    "assist_tackle_3_player_id",
    "lateral_kickoff_returner_player_name",
    "kickoff_returner_player_name",
    "solo_tackle_2_player_name",
]

In [45]:
drop_columns3=[
    "drive_inside20",
    "penalty",
    "penalty_yards",
    "penalty_type",
    "drive_play_id_ended",
    "drive_play_id_started",
    "first_down_penalty",
    "fourth_down_converted",
    "fourth_down_failed",
    "fumble_lost",
    "fumble_recovery_1_player_id",
    "fumble_recovery_1_player_name",
    "fumble_recovery_1_team",
    "fumble_recovery_1_yards",
    "fumble_recovery_2_player_id",
    "fumble_recovery_2_player_name",
    "fumble_recovery_2_team",
    "fumble_recovery_2_yards",
    "pass_attempt",
    "qb_epa",
    "rush_attempt",
    "safety",
    "series_result",
    "series_success",
    'third_down_converted',
    'third_down_failed',
    'complete_pass',
    "incomplete_pass",
    "sack",
    "tackled_for_loss",
    "half_seconds_remaining",
    "game_half",
    #"weather",
    "air_yards",
    "yards_after_catch"
    
]

In [46]:
# drop columns
combined_df.drop(drop_columns1, axis=1, inplace=True)
combined_df.drop(drop_columns2, axis=1, inplace=True)
combined_df.drop(drop_columns3, axis=1, inplace=True)

In [59]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(combined_df.head(10))
pd.reset_option('display.max_columns')
pd.reset_option('display.max_rows')

,posteam,posteam_type,yardline_100,game_date,game_seconds_remaining,down,goal_to_go,ydstogo,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_scramble,air_yards,yards_after_catch,score_differential,td_prob,ep,epa,wp,wpa,first_down_rush,first_down_pass,interception,qb_hit,fumble,cp,series,weather,div_game,roof
0,PHI,home,77.0,1999-09-12,3600.0,1.0,0.0,10,(15:00) D.Pederson pass incomplete to D.Staley...,pass,0.0,0,0,1.0,0,NaN,NaN,0.0,0.342473,0.764706,-0.561568,0.546969,-0.018228,0.0,0.0,0.0,0.0,0.0,NaN,1,NaN,1,outdoors
1,PHI,home,77.0,1999-09-12,3600.0,2.0,0.0,10,(15:00) D.Staley up the middle to PHI 24 for 1...,run,1.0,0,0,0.0,0,NaN,NaN,0.0,0.310568,0.203138,-0.641717,0.528741,-0.030422,0.0,0.0,0.0,0.0,0.0,NaN,1,NaN,1,outdoors
2,PHI,home,81.0,1999-09-12,3600.0,3.0,0.0,14,(15:00) D.Pederson pass to B.Finneran to PHI 2...,pass,10.0,0,0,1.0,0,NaN,NaN,0.0,0.224716,-1.161881,0.212661,0.490872,-0.006659,0.0,0.0,0.0,0.0,0.0,NaN,1,NaN,1,outdoors
3,ARI,away,81.0,1999-09-12,3487.0,1.0,0.0,10,(13:07) A.Murrell right end to ARI 20 for 1 ya...,run,1.0,0,0,0.0,0,NaN,NaN,0.0,0.279041,-0.078607,-0.392644,0.482290,-0.010499,0.0,0.0,0.0,0.0,0.0,NaN,2,NaN,1,outdoors
4,ARI,away,80.0,1999-09-12,3487.0,2.0,0.0,9,(13:07) A.Murrell up the middle to ARI 24 for ...,run,4.0,0,0,0.0,0,NaN,NaN,0.0,0.254127,-0.471251,-0.302914,0.471791,-0.010026,0.0,0.0,0.0,0.0,0.0,NaN,2,NaN,1,outdoors
5,ARI,away,76.0,1999-09-12,3487.0,3.0,0.0,5,(13:07) J.Plummer pass incomplete to F.Sanders.,pass,0.0,0,0,1.0,0,NaN,NaN,0.0,0.234744,-0.774165,-1.338036,0.461765,-0.038263,0.0,0.0,0.0,0.0,0.0,NaN,2,NaN,1,outdoors
6,PHI,home,59.0,1999-09-12,3388.0,1.0,0.0,10,(11:28) D.Staley left tackle to PHI 48 for 7 y...,run,7.0,0,0,0.0,0,NaN,NaN,0.0,0.402300,2.043952,0.361135,0.572573,0.013275,0.0,0.0,0.0,0.0,0.0,NaN,3,NaN,1,outdoors
7,PHI,home,52.0,1999-09-12,3388.0,2.0,0.0,3,(11:28) D.Pederson pass to C.Johnson pushed ob...,pass,15.0,0,0,1.0,0,NaN,NaN,0.0,0.427834,2.405087,0.999928,0.585848,0.046527,0.0,1.0,0.0,0.0,0.0,NaN,3,NaN,1,outdoors
8,PHI,home,37.0,1999-09-12,3388.0,1.0,0.0,10,(11:28) K.Turner up the middle to ARI 36 for 1...,run,1.0,0,0,0.0,0,NaN,NaN,0.0,0.462421,3.405015,-0.405522,0.632376,-0.014583,0.0,0.0,0.0,0.0,0.0,NaN,4,NaN,1,outdoors
9,PHI,home,36.0,1999-09-12,3388.0,2.0,0.0,9,(11:28) D.Pederson pass to D.Jells to ARI 30 f...,pass,6.0,0,0,1.0,0,NaN,NaN,0.0,0.416372,2.999493,0.026725,0.617792,-0.008916,0.0,0.0,0.0,0.0,0.0,NaN,4,NaN,1,outdoors


In [56]:
combined_df[combined_df["cp"].isna()].shape[0]/combined_df.shape[0]

0.6190236178717401

In [54]:
combined_df.shape

(799564, 33)

In [64]:
combined_df["game_date"][0] > "2009-08-01"

False

In [113]:
for year in list(range(1999,2024)):
    print(f"{year}: {combined_df[(combined_df['play_type']=='pass') & (combined_df['yards_gained']>0) &~(combined_df['cp'].isna()) & (combined_df['game_date'] >=str(year)) & (combined_df['game_date'] <=str(year+1))].shape}")

1999: (0, 33)
2000: (0, 33)
2001: (0, 33)
2002: (0, 33)
2003: (0, 33)
2004: (0, 33)
2005: (0, 33)
2006: (8638, 33)
2007: (9993, 33)
2008: (9850, 33)
2009: (9489, 33)
2010: (10210, 33)
2011: (10118, 33)
2012: (11234, 33)
2013: (10753, 33)
2014: (10798, 33)
2015: (10479, 33)
2016: (11173, 33)
2017: (11162, 33)
2018: (11081, 33)
2019: (10924, 33)
2020: (10690, 33)
2021: (11261, 33)
2022: (11540, 33)
2023: (6709, 33)


In [107]:
df=combined_df_copy[:10000]

In [111]:
for year in list(range(1999,2024)):
    print(f"{year}: {combined_df[(combined_df['play_type'] == 'pass') & (combined_df['yards_gained'] > 0) & (~combined_df['weather'].isna()) & (combined_df['game_date'] >= str(year)) & (combined_df['game_date'] < str(year + 1))].shape}")

1999: (0, 33)
2000: (0, 33)
2001: (8506, 33)
2002: (10612, 33)
2003: (9537, 33)
2004: (8977, 33)
2005: (1530, 33)
2006: (104, 33)
2007: (169, 33)
2008: (108, 33)
2009: (73, 33)
2010: (62, 33)
2011: (56, 33)
2012: (112, 33)
2013: (32, 33)
2014: (66, 33)
2015: (10241, 33)
2016: (11212, 33)
2017: (11185, 33)
2018: (11126, 33)
2019: (10962, 33)
2020: (10729, 33)
2021: (11312, 33)
2022: (11584, 33)
2023: (6548, 33)


In [121]:
# detect NAs
for column in combined_df.columns:
    print(f"{column}: {sum(combined_df[column].isna())}")

posteam: 1
posteam_type: 0
yardline_100: 0
game_date: 0
game_seconds_remaining: 1
down: 2033
goal_to_go: 2
ydstogo: 0
desc: 0
play_type: 0
yards_gained: 0
shotgun: 0
no_huddle: 0
qb_dropback: 0
qb_scramble: 0
air_yards: 488367
yards_after_catch: 604430
score_differential: 0
td_prob: 0
ep: 11
epa: 13
wp: 11
wpa: 15
first_down_rush: 0
first_down_pass: 0
interception: 0
qb_hit: 0
fumble: 0
cp: 494949
series: 0
weather: 385043
div_game: 0
roof: 3033
